In [3]:
# step1:
import pandas as pd
file_path = 'cme_data_2024_onwards.csv'  # Update path if needed
df = pd.read_csv(file_path)
df['halo?'] = df['halo?'].astype(str).str.strip()
velocity_columns = ['v', 'minv', 'maxv']
for col in velocity_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')
filtered_df = df[
    (df['halo?'].isin(['II', 'III', 'IV'])) &  # Halo or Partial Halo
    (df['v'] > 400) &                          # Median speed > 400 km/s
    (df['maxv'] > df['minv'])  &        # Positive acceleration
    (df['pa'].between(90, 270))                # Earth-directed angle range
]

filtered_df.to_csv("likely_earth_directed_cmes.csv", index=False)
print("Filtered CMEs saved to 'likely_earth_directed_cmes.csv'")
print(filtered_df[['CME', 't0', 'pa', 'da', 'v', 'minv', 'maxv', 'halo?']])


Filtered CMEs saved to 'likely_earth_directed_cmes.csv'
      CME               t0   pa   da     v  minv  maxv halo?
8       9   1/2/2024 11:12  248  118   558   249   976    II
47     48   1/9/2024 15:24  144  126   504   192   844    II
77     78  1/12/2024 22:12  127   96   496   176   919    II
91     92  1/14/2024 12:00  265  108   762   216  1249    II
120   121   1/20/2024 9:12  125  132   651   202  1275    II
...   ...              ...  ...  ...   ...   ...   ...   ...
2464   80   3/19/2025 8:12  183  152  1349   351  2016    II
2524  140  3/28/2025 15:24  144  360  1644   328  1953    IV
2622   76   4/13/2025 7:00  175  228   416   150   637   III
2650  104   4/19/2025 0:00  158   98   512   218   801    II
2866  153  5/30/2025 23:48  188  230   822   206  1967   III

[90 rows x 8 columns]


In [8]:
import pandas as pd
import requests
from datetime import datetime

# Step 1: Download ICME table
url = "https://izw1.caltech.edu/ACE/ASC/DATA/level3/icmetable2.htm"
response = requests.get(url)
response.raise_for_status()
tables = pd.read_html(response.text, header=1)

# Step 2: Load main table
df = tables[0]
df.columns = [str(col).strip() for col in df.columns]

# Step 3: Rename relevant columns
df.rename(columns={
    'LASCO CME Y/M/D (UT) (o)': 'LASCO_CME_UT',
    'Disturbance Y/M/D (UT) (a)': 'Disturbance_UT',
    'ICME Plasma/Field Start, End Y/M/D (UT) (b)': 'ICME_start_UT',
    'V_transit (km/s) (n)': 'V_transit_km_s',
}, inplace=True)

# Step 4: Robust date extraction function
def extract_first_date(text):
    if pd.isna(text) or not isinstance(text, str):
        return None
    try:
        # Extract first date-time like string
        parts = text.split(',')[0].strip().split()
        if len(parts) < 2:
            return None
        date_part, time_part = parts[0], parts[1][:4]  # Keep only HHMM
        return datetime.strptime(date_part + ' ' + time_part, '%Y/%m/%d %H%M')
    except:
        return None

# Step 5: Apply parsing
df['LASCO_CME_UT_clean'] = df['LASCO_CME_UT'].apply(extract_first_date)

# Step 6: Filter for August 2024 onward
cutoff = datetime(2024, 8, 1)
filtered = df[df['LASCO_CME_UT_clean'] >= cutoff]

# Step 7: Show result
print(filtered[['LASCO_CME_UT_clean', 'Disturbance_UT', 'ICME_start_UT', 'V_transit_km_s']])

# Step 8: Save to CSV if needed
filtered.to_csv("CME_Events_Aug_2024_Onward.csv", index=False)


     LASCO_CME_UT_clean   Disturbance_UT    ICME_start_UT V_transit_km_s
634 2024-08-07 03:24:00  2024/08/10 1250  2024/08/11 1200          >=510
635 2024-08-14 08:24:00  2024/08/17 1423  2024/08/17 1700            530
636 2024-08-23 02:00:00  2024/08/27 0841  2024/08/27 1600            400
640 2024-10-03 12:48:00  2024/10/06 0739  2024/10/06 1400            620
641 2024-10-03 20:36:00  2024/10/08 0000  2024/10/08 0000        420-450
642 2024-10-08 06:12:00  2024/10/10 1514  2024/10/10 2200       730-1120
644 2024-11-25 03:12:00  2024/11/29 0305  2024/11/29 1700        430-540
645 2024-12-15 01:25:00  2024/12/17 0517  2024/12/17 0800        800-850
646 2024-12-29 01:23:00  2024/12/31 1623  2025/01/01 1000        660-900
